In [1]:
from src.standarice_data import TenisModelHandler
import pandas as pd

In [5]:
# Cargar datos de entrenamiento
df_train = pd.read_csv('../data\input\scraped\df_games.csv', delimiter='|')

# Crear instancia de la clase Preprocessing
preprocessor = TenisModelHandler(df_train, models_path='../models')

# Preprocesar datos de entrenamiento
preprocessor.preprocessing_train()

# Entrenar modelos
models_to_train = ['xgboost']
preprocessor.train_models(models_to_train)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Julián Pérez\AppData\Local\Temp\ipykernel_19296\2379427704.py:2: SyntaxWarning: invalid escape sequence '\i'
  df_train = pd.read_csv('../data\input\scraped\df_games.csv', delimiter='|')
g:\dev\tennis_bet\code\src\standarice_data.py:304: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  player_data['weight'].fillna(mean_values['weight'], inplace=True)
g:\dev\tennis_bet\code\src\standarice_data.py:305: FutureWarning: A value

Entrenando xgboost...


g:\dev\tennis_bet\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:13:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores hiperparámetros xgboost: {'model__learning_rate': 0.1, 'model__max_depth': 7, 'model__n_estimators': 200}
Accuracy en entrenamiento xgboost: 0.7502
Accuracy en prueba para xgboost: 0.7512
Reporte de clasificación para xgboost:
              precision    recall  f1-score   support

           0       0.75      0.75      0.75      8420
           1       0.75      0.75      0.75      8472

    accuracy                           0.75     16892
   macro avg       0.75      0.75      0.75     16892
weighted avg       0.75      0.75      0.75     16892



In [26]:
# Cargar nuevos datos para predicción
df_new = pd.read_csv('../data/input/scraped/df_testing.csv', delimiter='|')
#df_new = pd.read_csv('../data/input/daily_dump/df_games_sep_test.csv', delimiter='|')

# Crear instancia de la clase Preprocessing
preprocessor = TenisModelHandler(df_new, models_path='../models')

# Generar predicciones con el modelo deseado
resultado = preprocessor.predict_values('xgboost')

# Mostrar el DataFrame con las predicciones
print(resultado.head())

             Location  pl1_bet  pl2_bet            pl1_name      pl1_flag  \
0             US Open     2.20     1.67           Draper J.  GreatBritain   
1             US Open     2.32     1.62           Tiafoe F.           USA   
3    Genoa challenger     1.69     2.09         Poljicak M.       Croatia   
4    Genoa challenger     1.31     3.27         Monteiro T.        Brazil   
6  Sevilla challenger     1.18     4.47  Carballes Baena R.         Spain   

   pl1_year_pro  pl1_weight  pl1_height pl1_hand  pl1_age  ...   pl2_flag  \
0        2017.0   80.000000  191.000000     left     22.0  ...  Australia   
1        2013.0   86.000000  188.000000    right     26.0  ...   Bulgaria   
3        2020.0   78.380551  184.629326    right     20.0  ...   Slovakia   
4        2008.0   78.000000  183.000000     left     30.0  ...    Lebanon   
6        2009.0   77.000000  183.000000    right     31.0  ...   Portugal   

  pl2_year_pro  pl2_weight  pl2_height  pl2_hand pl2_age  Surface      Dat

In [28]:
resultado["winner_prediction"].value_counts()

winner_prediction
pl1    130
pl2     64
Name: count, dtype: int64

In [29]:
130/(64+130)

0.6701030927835051

In [24]:
import pandas as pd

# Capital inicial
initial_capital = 100.0  # en USD

# Preferencia de riesgo (0 < risk_factor <= 1)
risk_factor = 0.9799

# Máximo de apuesta diaria como fracción del capital total
max_daily_bet_fraction = 0.8996 # Por ejemplo, máximo 20% del capital por día

max_bet_fraction_per_match = 0.4851

probability_rule = 0.9522

# Capital actual
bankroll = initial_capital

# Agrupar datos por fecha
dates = resultado['Date'].unique()

# Historial del capital
capital_history = [bankroll]

# Lista para almacenar detalles de las apuestas
bet_details = []

for date in dates:
    day_matches = resultado[resultado['Date'] == date]
    daily_bet_amount = 0
    daily_max_bet = bankroll * max_daily_bet_fraction
    bets = []
    
    for idx, match in day_matches.iterrows():

        if match["winner_prediction"] == "pl1":
            # Cálculo para el Jugador 1
            p1_win_prob = match['probability']
        else:
            p1_win_prob = 1 - match['probability']
            
        p1_loss_prob = 1 - p1_win_prob
        odds_p1 = match['pl1_bet']
        b1 = odds_p1 - 1
        f1 = (b1 * p1_win_prob - p1_loss_prob) / b1
        f1 = max(f1, 0) * risk_factor
        f1 = min(f1, 1)
        
        # Cálculo para el Jugador 2
        p2_win_prob = 1 - p1_win_prob
        p2_loss_prob = 1 - p2_win_prob
        odds_p2 = match['pl2_bet']
        b2 = odds_p2 - 1
        f2 = (b2 * p2_win_prob - p2_loss_prob) / b2
        f2 = max(f2, 0) * risk_factor
        f2 = min(f2, 1)
        
        # Decidir por qué jugador apostar
        if f1 > f2 and f1 > 0:
            f = f1
            odds = odds_p1
            player_bet_on = 'pl1'
            win_prob = p1_win_prob
        elif f2 > f1 and f2 > 0:
            f = f2
            odds = odds_p2
            player_bet_on = 'pl2'
            win_prob = p2_win_prob
        else:
            continue  # No apostar si no hay valor positivo de Kelly

        if win_prob <= probability_rule:
            continue 
        
        max_bet_amount_per_match = bankroll * max_bet_fraction_per_match

        # Calcular monto de la apuesta
        bet_amount = f * bankroll
        bet_amount = min(bet_amount, max_bet_amount_per_match)
        
        # Asegurar que no se exceda el máximo diario
        if daily_bet_amount + bet_amount > daily_max_bet:
            bet_amount = daily_max_bet - daily_bet_amount
            if bet_amount <= 0:
                continue
            f = bet_amount / bankroll  # Recalcular fracción de Kelly
        
        daily_bet_amount += bet_amount
        
        bets.append({
            'MatchIndex': idx,
            'Date': date,
            'PlayerBetOn': player_bet_on,
            'Odds': odds,
            'KellyFraction': f,
            'BetAmount': bet_amount,
            'WinProbability': win_prob,
            'Opponent': 'pl2' if player_bet_on == 'pl1' else 'pl1'
        })
    
    # Actualizar capital basado en los resultados
    for bet in bets:
        bet_amount = bet['BetAmount']
        idx = bet['MatchIndex']
        match = resultado.loc[idx]
        actual_winner = "pl1"
        bet['ActualWinner'] = actual_winner
        if actual_winner == bet['PlayerBetOn']:
            payout = bet_amount * bet['Odds']
            profit = payout - bet_amount
            bankroll += profit
            bet['Profit'] = profit
            bet['Result'] = 'Win'
        else:
            bankroll -= bet_amount
            bet['Profit'] = -bet_amount
            bet['Result'] = 'Loss'
        
        print(bankroll)

        # Añadir detalle de la apuesta a la lista principal
        bet_details.append(bet)
    
    capital_history.append(bankroll)

# Crear DataFrame con los detalles de las apuestas
bets_df = pd.DataFrame(bet_details)

# Mostrar tabla de resultados
print(bets_df[['Date', 'MatchIndex', 'PlayerBetOn', 'Opponent', 'Odds', 'BetAmount', 'Profit', 'Result']])

# Mostrar capital final
print(f"\nCapital final: {bankroll:.2f} USD")


101.91259590963126
102.05225070952439
104.45027970954891
105.73328125579336
106.74171288520049
108.81292908202491
109.12185467353284
111.42570750677436
111.61774284280209
112.82211508780358
113.93617082305079
116.50569392900158
118.68277911171552
122.13716008054112
        Date  MatchIndex PlayerBetOn Opponent  Odds  BetAmount    Profit  \
0   20240911         152         pl1      pl2  1.06  31.876598  1.912596   
1   20240912         292         pl1      pl2  1.02   6.982740  0.139655   
2   20240913         400         pl1      pl2  1.06  39.967150  2.398029   
3   20240914         455         pl1      pl2  1.06  21.383359  1.283002   
4   20240917         522         pl1      pl2  1.05  20.168633  1.008432   
5   20240922        1409         pl1      pl2  1.04  51.780405  2.071216   
6   20240922        1640         pl1      pl2  1.02  15.446280  0.308926   
7   20240922        1660         pl1      pl2  1.08  28.798160  2.303853   
8   20240923        1702         pl1      pl2  1.0

In [25]:
bets_df

,MatchIndex,Date,PlayerBetOn,Odds,KellyFraction,BetAmount,WinProbability,Opponent,ActualWinner,Profit,Result
0,152,20240911,pl1,1.06,0.318766,31.876598,0.961810,pl2,pl1,1.912596,Win
1,292,20240912,pl1,1.02,0.068517,6.982740,0.981763,pl2,pl1,0.139655,Win
2,400,20240913,pl1,1.06,0.391634,39.967150,0.966019,pl2,pl1,2.398029,Win
3,455,20240914,pl1,1.06,0.204723,21.383359,0.955222,pl2,pl1,1.283002,Win
4,522,20240917,pl1,1.05,0.190750,20.168633,0.961651,pl2,pl1,1.008432,Win
5,1409,20240922,pl1,1.04,0.514971,51.780405,0.981751,pl2,pl1,2.071216,Win
6,1640,20240922,pl1,1.02,0.144707,15.446280,0.983288,pl2,pl1,0.308926,Win
7,1660,20240922,pl1,1.08,0.269793,28.798160,0.967475,pl2,pl1,2.303853,Win
8,1702,20240923,pl1,1.02,0.086172,9.601767,0.982116,pl2,pl1,0.192035,Win
9,1909,20240924,pl1,1.04,0.269754,30.109306,0.972126,pl2,pl1,1.204372,Win
